In [21]:
import pandas as pd
import json
import os
import re
import text_unify
import docano_module

In [22]:
#list of exist labels
list_of_labels = ['type', 'subtype', 'name', 'code', 'brand', 'origin', 'material', 
                  'color', 'gender', 'numunit', 'size', 'pattern', 'form', 'target', 
                  'flavor', 'misc', 'attach', 'author', 'status']

target_labels = ['type', 'name', 'color', 'brand', 'material', 'form', 'pattern', 'numunit']
vn_target_labels = ['sản phẩm', 'tên gọi', 'màu sắc', 'nhãn hiệu', 'chất liệu', 'hình dáng', 'họa tiết', 'kích thước']

label_mapping = {'code': 'name', 'size': 'numunit', 'subtype': 'misc'}
vnlabel_mapping = {'type': 'sản phẩm', 'name': "tên gọi", "brand": "nhãn hiệu", "origin": "xuất xứ",
                  "material": "chất liệu", "color": "màu sắc", "gender": "giới tính", 
                   "numunit": "kích thước", "pattern" : "họa tiết", "form": "hình dáng",
                  "target": "đối tượng", "flavor": "mùi vị", "misc": "đặc điểm khác", 
                   "attach": "đính kèm", "author": "tác giả", "status": "trạng thái"}

In [23]:
def unify_text(text):
    ntext = re.sub(r"[\.,\?]+$-", "", text)
    ntext = ntext.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("[", " ").replace("]", " ").replace(",", " ")
    
    ntext = ' '.join(ntext.split())
    ntext = ntext.lower()
    ntext = text_unify.unify(ntext)
    return ntext

In [24]:
# old structure - not merged per catelv1:
## fname: have cate_id_lv2
## title: [cate_id_lv2]( )title
# new structure - merged per catelv1
## fname: have cate_id_lv1
## title: [index]( )[cate_id_lv2]( )title
def get_cate_id_from_raw_title_data(title):
    ntitle = title.replace('[', ' ').replace(']', ' ')
    titles = ntitle.split()
    num1 = titles[0]
    num2 = titles[1]
    if num1.isnumeric() and num2.isnumeric(): 
        return int(num2)
    return num1.isnumeric() and int(num1) or 0

In [25]:
def get_raw_title(title):
    ntitle = title.replace('[', ' ').replace(']', ' ')
    titles = ntitle.split()
    num1 = titles[0]
    num2 = titles[1]
    if num1.isnumeric() and num2.isnumeric(): 
        ntitle = ' '.join(titles[2:])
    else:
        ntitle = ' '.join(titles[1:])
    return ntitle

In [26]:
# labels = [[sIndex, eIndex, label], ...]
def filter_label(labels):
    nlabels = []
    for sIndex, eIndex, label in labels:
        label = unify_text(label)
        if label in label_mapping:
            label = label_mapping[label]
        if label in vnlabel_mapping:
            label = vnlabel_mapping[label]
        if label in vn_target_labels:
            nlabels.append([sIndex, eIndex, label])
            
    return nlabels

### Main thread

In [27]:
raw_data = open("../data/raw_data.jsonl", "r", encoding = 'utf-8').readlines()

In [30]:
def json2df(raw_data, has_cate_id=True):
    ids = []
    cate_ids = []
    words = []
    labels = []
    tags = []
    
    for id, line in enumerate(raw_data):
        fully_annot = docano_module.doccano2fully(line)
        fully_annot = json.loads(fully_annot)
        content = fully_annot['content']
        if content == '':
            continue
        
        if has_cate_id:
            cate_id = str(get_cate_id_from_raw_title_data(content))
            content = get_raw_title(content)
        else:
            cate_id = ''
            
        annotations = fully_annot['annotation']
        for annot in annotations:
            label = annot['label']
            text = annot['points'][0]['text']
            if (text in content) and (text.split() != ''):
                ids.append(id)
                cate_ids.append(cate_id)
                words.append(unify_text(text))
                labels.append(label[0])
                tags.append(label[1] + '-' + label[0])
    
    df = pd.DataFrame({'id': ids, 'cate_id': cate_ids, 'word': words, 'label': labels, 'tag': tags})
    return df
                


In [29]:
df = json2df(raw_data)
df.to_csv('../data/product_title_dataset.csv', index=False)